In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
coordinates = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
coordinates = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for coord in coordinates:
    city = citipy.nearest_city(coord[0], coord[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

632

In [6]:
cities

['vaini',
 'aquiraz',
 'pau brasil',
 'broome',
 'lagoa',
 'suoyarvi',
 'punta arenas',
 'georgetown',
 'ribeira grande',
 'taolanaro',
 'vardo',
 'yanan',
 'grand river south east',
 'albany',
 'thompson',
 'bluff',
 'balimo',
 'avarua',
 'taguatinga',
 'saskylakh',
 'tiksi',
 'komsomolskiy',
 'wajima',
 'makakilo city',
 'kapoeta',
 'kamenskoye',
 'severo-yeniseyskiy',
 'lata',
 'iqaluit',
 'panguna',
 'illoqqortoormiut',
 'jamestown',
 'lolua',
 'belushya guba',
 'severo-kurilsk',
 'fairbanks',
 'madison',
 'barrow',
 'kavieng',
 'vostok',
 'salalah',
 'mahebourg',
 'tsihombe',
 'hermanus',
 'la asuncion',
 'carnarvon',
 'te anau',
 'venado tuerto',
 'ushuaia',
 'hobart',
 'farap',
 'bredasdorp',
 'zaraza',
 'tautira',
 'east london',
 'yaan',
 'siocon',
 'new norfolk',
 'lebu',
 'bridlington',
 'rikitea',
 'ahipara',
 'yellowknife',
 'mataura',
 'saint george',
 'sao filipe',
 'buta',
 'port elizabeth',
 'mayo',
 'upernavik',
 'piacabucu',
 'airai',
 'kilakarai',
 'nikolskoye',
 'p